In [ ]:
import torch
import numpy as np
import pandas as pd

def epid_model(population, R, initial_newly_infected, alpha, device, dtype):
        n = R.size()[0]

        df_si_dd = pd.read_csv('data/si_and_dd_covid19model.csv', sep=',', decimal='.', encoding='utf-8', dtype={'serial_interval': np.float64, 'delay_distr': np.float64})
        si = torch.tensor(df_si_dd['serial_interval'][0:n],dtype=dtype, device=device)
        dd =  torch.tensor(df_si_dd['delay_distr'][0:n],dtype=dtype, device=device)
        # Seed initial Infections
        newly_infected = torch.zeros(n, dtype=dtype, device=device)
        # number of newly infected
        cumulative_infected = torch.zeros(n, dtype=dtype, device=device)
        #cumulative number of infected
        immune = torch.zeros(n,dtype=dtype, device=device)
        St = torch.ones(n,dtype=dtype, device=device)  # fraction of susceptible population
        effectively_infectious = torch.zeros(n, dtype=dtype, device=device)
        cero = torch.tensor(0.,dtype=dtype, device=device)

        num_init = initial_newly_infected.size
        newly_infected[0:num_init] = torch.tensor(initial_newly_infected, dtype=dtype, device=device)
        cumulative_infected[1:num_init] = torch.cumsum(newly_infected[0:num_init - 1].clone(), dim=0)

        immune[0:num_init] = cumulative_infected[0:num_init].clone() #+ vaccine_inmune[0:num_impute].clone()
        # TODO: Add vaccine

        for t in range(num_init, n):
                # Update cumulative newly_infected
                cumulative_infected[t] = cumulative_infected[t - 1] + newly_infected[t - 1]
                immune[t] = cumulative_infected[t] #+ vaccine_inmune[t]
                # Adjusts for portion of pop that are susceptible
                St[t] = torch.maximum(population - immune[t], cero) / population
                # effective number of infectous individuals
                ni_temp = newly_infected[:t].view(1, 1, -1).clone()
                si_temp = torch.flip(si, (0,))[-t:].view(1, 1, -1)
                effectively_infectious[t] = torch.nn.functional.conv1d(ni_temp, si_temp)

                newly_infected[t] = St[t].clone() * R[t] * effectively_infectious[t].clone()

        # calc hospitalizations
        # Initialize expected_daily_hospit
        expected_daily_hospit = torch.zeros(n, dtype=dtype, device=device)  # expected number of daily hospitalizations

        # calculate expected number of hospitalizations
        expected_daily_hospit[0] = (1e-15) * newly_infected[0]
        for t in range(1, n):
                ni_temp = newly_infected[:t].view(1, 1, -1)
                dd_temp = torch.flip(dd, (0,))[-t - 1:-1].view(1, 1, -1)
                expected_daily_hospit[t] = alpha * torch.nn.functional.conv1d(ni_temp, dd_temp)

        return {'St': St, 'newly_infected': newly_infected, 'cumulative': cumulative_infected, 'hospitalization': expected_daily_hospit}